In [2]:
!pip install yt-dlp openai python-dotenv ipykernel

In [4]:

import os
import subprocess
import openai
from openai import OpenAI
import uuid
from dotenv import load_dotenv

# Load environment variables
load_dotenv(dotenv_path="../env_file/.env")


True

In [6]:

def download_audio(video_url):
    try:
        unique_id = str(uuid.uuid4())
        output_filename = f"downloaded_audio_{unique_id}.mp3"
        command = [
            'yt-dlp',
            '-f', 'bestaudio',
            '--extract-audio',
            '--audio-format', 'mp3',
            '--ffmpeg-location', '/opt/homebrew/bin/ffmpeg',
            '--output', output_filename.replace(".mp3", ".%(ext)s"),
            video_url
        ]
        subprocess.run(command, check=True)
        return output_filename
    except Exception as e:
        print(f"Error downloading audio: {e}")
        return None


In [8]:
def transcribe_audio(audio_path):
    try:
        client = OpenAI()
        with open(audio_path, "rb") as audio_file:
            transcript = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file,
                response_format="verbose_json"
            )
        text = transcript.text
        segments = transcript.segments
        return text, segments
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return None, None


In [10]:
def format_segments_with_timestamps(segments):
    formatted = []
    for seg in segments:
        start = int(seg.start)
        minutes = start // 60
        seconds = start % 60
        timestamp = f"{minutes}:{seconds:02d}"
        formatted.append(f"[{timestamp}] {seg.text}")
    return "\n".join(formatted)

In [12]:
def summarize_text(transcript, segments, mode="basic"):
    prompts = {
        "basic": f"Summarize the following transcript:\n\n{transcript}",
        "bullets": f"Summarize the following transcript into concise bullet points:\n\n{transcript}",
        "quotes": f"Extract 5-10 of the most compelling or insightful quotes from this transcript. Include timestamps:\n\n{format_segments_with_timestamps(segments)}",
        "insights": f"Summarize the following transcript by extracting the key insights and implications. Be concise but thoughtful:\n\n{transcript}"
    }

    prompt = prompts.get(mode, prompts["basic"])

    try:
        client = OpenAI()
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=700
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error generating summary: {e}")
        return None


In [16]:

video_url = input("Enter YouTube URL: ").strip()
mode = input("Choose summary style (basic, bullets, quotes, insights): ").strip().lower()
show_transcript = input("Would you like to see the full transcript first? (yes/no): ").strip().lower()

audio_file = download_audio(video_url)

if audio_file:
    transcript_text, segments = transcribe_audio(audio_file)
    if transcript_text:
        if show_transcript == "yes":
            print("\n--- Transcript fetched ---\n")
            print(transcript_text)

        print("\n--- Summary ---\n")
        print(summarize_text(transcript_text, segments, mode))
    else:
        print("Transcription failed.")
else:
    print("Audio download failed.")


Enter YouTube URL:  https://www.youtube.com/watch?v=2Zbg5iuDxOg
Choose summary style (basic, bullets, quotes, insights):  quotes
Would you like to see the full transcript first? (yes/no):  yes


[youtube] Extracting URL: https://www.youtube.com/watch?v=2Zbg5iuDxOg
[youtube] 2Zbg5iuDxOg: Downloading webpage
[youtube] 2Zbg5iuDxOg: Downloading tv client config
[youtube] 2Zbg5iuDxOg: Downloading tv player API JSON
[youtube] 2Zbg5iuDxOg: Downloading ios player API JSON


[youtube] 2Zbg5iuDxOg: Downloading m3u8 information
[info] 2Zbg5iuDxOg: Downloading 1 format(s): 234
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 104
[download] Destination: downloaded_audio_f3982cbb-8a0e-4d8c-b32f-c5d86daa8205.mp4
[download] 100% of    8.19MiB in 00:00:16 at 520.53KiB/s                 
[ExtractAudio] Destination: downloaded_audio_f3982cbb-8a0e-4d8c-b32f-c5d86daa8205.mp3
Deleting original file downloaded_audio_f3982cbb-8a0e-4d8c-b32f-c5d86daa8205.mp4 (pass -k to keep)

--- Transcript fetched ---

AI has opened a new continent, and humanity is setting foot on its shores. Since humans have walked the planet, we've always been the most impressive, intelligent beings. But today, AI is forcing us to ask the question, what does it mean to be human? To live a flourishing human life? Major moments in science and technology shed new light on long-standing philosophical questions. Copernicus dislodged us from our central position in the cosmos. Charles Dar